In [2]:
import duckdb
import pandas as pd

In [3]:
sql_query = '''
select * from gold_vehicles_metrics
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,registration_state,ticket_count
0,NJ,9258
1,PA,3514
2,FL,2414
3,CT,1787
4,GA,840
...,...,...
57,AB,1
58,BC,1
59,NB,1
60,PE,1


In [4]:
sql_query_import_1 = '''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT *
FROM read_csv_auto(
  'data/dof_parking_violation_codes.csv',
  normalize_names=True
  )
'''

sql_query_import_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto(
  'data/parking_violations_issued_fiscal_year_2023_sample.csv',
  normalize_names=True
  )
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2)

In [5]:
sql_query = '''
show tables
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,bronze_parking_violation
1,bronze_parking_violation_codes
2,bronze_parking_violations
3,first_model
4,gold_ticket_metrics
5,gold_vehicles_metrics
6,parking_violation_codes
7,parking_violations_2023
8,ref_model
9,silver_parking_violation_codes
